# Проект "GoFast"
Нам передали данные о **некоторых пользователях** из нескольких городов, а также об их **поездках**. Нам необходимо проанализировать данные и проверить некоторые гипотезы, которые могут помочь бизнесу вырасти.
Чтобы совершать поездки по городу, пользователи сервиса **GoFast** пользуются мобильным приложением с двумя опциями - с подпиской и без неё.
**Цель проекта** - оценить являются ли пользователи с подпиской более выгодными для компании.
В основных данных есть информация о **пользователях, их поездках и подписках**. 

**Пользователи** — `users_go.csv`

* **user_id** -	уникальный идентификатор пользователя
* **name** - имя пользователя
* **age** -	возраст
* **city** - город
* **subscription_type**	- тип подписки (free, ultra)

**Поездки** — `rides_go.csv`

* **user_id** - уникальный идентификатор пользователя
* **distance** - расстояние, которое пользователь проехал в текущей сессии (в метрах)
* **duration** - продолжительность сессии (в минутах) — время с того момента, как пользователь нажал кнопку «Начать поездку» до момента, как он нажал кнопку «Завершить поездку»
* **date** - дата совершения поездки

**Подписки** — `subscriptions_go.csv`

* **subscription_type** - тип подписки
* **minute_price** - стоимость одной минуты поездки по данной подписке
* **start_ride_price** - стоимость начала поездки
* **subscription_fee** - стоимость ежемесячного платежа

**План исследования:**

* Загрузка данных
* Предобработка данных
* Исследовательский анализ данных
* Объединение данных
* Подсчёт выручки
* Проверка гипотез
* Распределения

In [ ]:
import pandas as pd #импортируем все библиотеки и/или методы и функции, которые понадобятся нам в исследовании
import matplotlib.pyplot as plt
from scipy import stats as st 
from math import sqrt
import numpy as np

# Шаг 1. Загрузка данных

In [ ]:
users = pd.read_csv('/datasets/users_go.csv') #считываем данные и записываем их в датафреймы
rides = pd.read_csv('/datasets/rides_go.csv')
subscriptions = pd.read_csv('/datasets/subscriptions_go.csv')

In [ ]:
users.head() #изучаем первые строки датафрейма users

In [ ]:
rides.head() #изучаем первые строки датафрейма rides

In [ ]:
subscriptions.head() #изучаем первые строки датафрейма subscriptions

In [ ]:
users.info() #изучаем общую информацию датафрейма users

In [ ]:
rides.info() #изучаем общую информацию датафрейма rides

In [ ]:
subscriptions.info() #изучаем общую информацию датафрейма subscriptions

## Промежуточный вывод
Мы считали CSV-файлы с данными с помощью библиотеки **pandas** и сохранили их в датафреймы. Выведили первые строки каждого набора данных и изучили общую информацию о каждом датафрейме. В датафрейме **users** находятся данные о имени, возрасте, городе и типе подписки для каждого уникального идентификатора пользователя. В датафрейме **rides** находятся данные о расстоянии, длительности и дате поездки для каждого уникального идентификатора пользователя. В датафрейме **subscriptions** находятся данные о цене за минуту поездки, цене старта поездки, цене подписки для каждого типа подписки.

# Шаг 2. Предобработка данных

In [ ]:
rides['date'] = pd.to_datetime(rides['date'], format='%Y-%m-%d') #переводим тип данных в столбце date в datetimeи

In [ ]:
rides['date'].min(), rides['date'].max() 
#поверяем что данные представлены за один календарный год чтобы убидеться что мы отсортируем данные корректно

In [ ]:
rides['month'] = pd.DatetimeIndex(rides['date']).month #создаем новый столбец с номером месяца на основе столбца date.

In [ ]:
users.isna().sum() #проверяем наличие пропусков в данных датафрейма users

In [ ]:
rides.isna().sum() #проверяем наличие пропусков в данных датафрейма rides

In [ ]:
subscriptions.isna().sum() #проверяем наличие пропусков в данных датафрейма subscriptions

In [ ]:
users.duplicated().sum() #проверяем наличие дубликатов в данных датафрейма users

In [ ]:
rides.duplicated().sum() #проверяем наличие дубликатов в данных датафрейма rides

In [ ]:
subscriptions.duplicated().sum() #проверяем наличие дубликатов в данных датафрейма subscriptions

In [ ]:
users = users.drop_duplicates().reset_index(drop=True) #обрабатываем дубликаты в датафрейме users

## Промежуточный вывод 
Мы привели столбец **date** датафрейма **rides** к необходимому типу данных - **datetime**. Также создали новый столбец **month** на основе данных столбца **date**. Пропуски в данных найдены не были. Дубликаты были найдеты только в датафрейме **users** в количестве 31 и удалили их, так как их мало и они выглядят как баг системы. 

# Шаг 3. Исследовательский анализ данных

In [ ]:
#строим сводную таблицу частоты встречаемости городов
users_city_count = users.groupby(['city']).agg({'user_id' : 'count'}).sort_values(by='user_id')
users_city_count

In [ ]:
users_city_count.plot(title='Частота встречаемости городов', kind='barh', xlabel='Города') #визуализируем частоту встречаемости городов
plt.show()

Города распределены практически равномерно, данных по каждому городу **от 168 до 219 ед.** Наибольшее количество поездок приходится на **Пятигорск**, наименьшее - на **Москву**)

In [ ]:
subscription_type_count = users.pivot_table(index='subscription_type', values='user_id', aggfunc='count') 
#строим сводную таблицу соотношения типов подписки
subscription_type_count

In [ ]:
subscription_type_count.plot(title='Соотношение пользователей с подпиской и без подписки', kind='pie', subplots=True, autopct='%1.2f%%')
plt.show() #визуализируем соотношение пользователей с подпиской и без подписки

Количество подписчиков сервиса около **700 ед.**, пользователей без подписки - немного больше **800 ед.**, они находятся в соотношении **45.27 % к 54.43 %** соответственно.

In [ ]:
age_count = users.pivot_table(index='age', values='user_id', aggfunc='count') 
#строим сводную таблицу возраста пользователей
age_count

In [ ]:
age_count.reset_index().boxplot()
plt.show() #визуализируем возраст пользователей
age_count.reset_index().describe()

Можно увидеть что большая часть значений возраста находится в области от **20 до 30**, среднее значение - **26**, минимальное - **12**, максимальное - **43**.

In [ ]:
len(rides['distance'].unique()) #оцениваем объём уникальных значений в столбце distance

In [ ]:
rides['distance'].plot(title='Расстояние, которое пользователь преодолел за одну поездку', kind='hist', grid=True) 
#визуализируем расстояние, которое пользователь преодолел за одну поездку
plt.show()

Наибольшее количество поездок пользователей были на расстояние **3000 - 3500 м**, наименьшее - на расстояние более **5000 м**)

In [ ]:
len(rides['duration'].unique()) #оцениваем объём уникальных значений в столбце duration

In [ ]:
rides['duration'].plot(title='Продолжительность поездок', kind='hist', grid=True)  #визуализируем продолжительность поездок
plt.show()

Наибольшее количество поездок пользователей были длительностью **17-20 минут**, наименьшее - длительностью **до 5 минут и более 33 минут**

## Промежуточный вывод 
На данном этапе нами были описаны и визуализированы следующие данные:
* частота встречаемости городов (города распределены практически равномерно, данных по каждому городу **от 168 до 219 ед.** Наибольшее количество поездок приходится на **Пятигорск**, наименьшее - на **Москву**)
* соотношение пользователей с подпиской и без подписки (количество подписчиков сервиса около **700 ед.**, пользователей без подписки - немного больше **800 ед.**, они находятся в соотношении **45.27 % к 54.43 %** соответственно.)
* возраст пользователей (можно увидеть что большая часть значений возраста находится в области от **20 до 30**, среднее значение - **26**, минимальное - **12**, максимальное - **43**.)
* расстояние, которое пользователь преодолел за одну поездку (наибольшее количество поездок пользователей были на расстояние **3000 - 3500 м**, наименьшее - на расстояние более **5000 м**)
* продолжительность поездок (наибольшее количество поездок пользователей были длительностью **17-20 минут**, наименьшее - длительностью **до 5 минут и более 33 минут**)

# Шаг 4. Объединение данных

In [ ]:
data = users.merge(right=rides, on='user_id') #соединяем датафреймы users и rides в новый датафрейм

In [ ]:
data = data.merge(right=subscriptions, on='subscription_type', how='right') 
#присоединяем к получившемуся датафрейму датафрейм subscriptions

In [ ]:
data.info() #оцениваем размерность итогового датафрейма

In [ ]:
data.isna().sum() #проверяем объединённый датасет на пропуски

In [ ]:
data_subs = data[data['subscription_type'] == 'ultra'] #создаём датафрейм для пользователей с подписками

In [ ]:
data_unsubs = data[data['subscription_type'] == 'free'] #создаём датафрейм для пользователей без подписок

In [ ]:
data_subs['distance'].plot(title='Растояние поездок для людей с подпиской', kind='hist', grid=True)
#визуализируем расстояние поездок для людей с подпиской
plt.show()
data_unsubs['distance'].plot(title='Растояние поездок для людей без подписок', kind='hist', grid=True)
#визуализируем расстояние поездок для людей без подписок
plt.show()

In [ ]:
data_subs['duration'].plot(title='Длительность поездок для людей с подпиской', kind='hist', grid=True)
#визуализируем длительность поездок для людей с подпиской
plt.show()
data_unsubs['duration'].plot(title='Длительность поездок для людей без подписки', kind='hist', grid=True)
#визуализируем длительность поездок для людей без подписки
plt.show()

## Промежуточный вывод
Мы объединили данные о пользователях, поездках и подписках в один датафрейм, воспользовавшись методом `merge()`. На основе этого датафрейма создали два новых - с данными о подписчиках и данными о пользователях без подписки, а также визуализировали информацию о расстоянии и времени поездок для пользователей обеих категорий. Самая частая длительность поездок подписчиков - **15-20 минут**, самая редкая - **до 7 минут и более 33 минут**. Самое частое расстояние поездок подписчиков - **3000-3500 м**, самое редкое - **до 2000 м и более 4500 м**. Самая частая длительность поездок пользователей без подписки - **14-21 минута**, самая редкая - **до 4 минут и более 31 минуты**. Самое частое расстояние поездок подписчиков - **2000-3500 м**, самое редкое - **более 5700 м**. 
Для обоих типов пользователей есть провисы в первой половине графика, но у пользователей с подпиской он находится на значении **1200-1800 м**, у пользователей без подписки - на значении **700-1500 м**. Относительно длительности - график для людей без подписки приближен к распределению по параболе, а графиц для людей с подпиской смещён влево.

# Шаг 5. Подсчёт выручки

In [ ]:
data['duration'] = np.ceil(data['duration']) #предварительно округляем данные по расстоянию в исходной таблице
data

In [ ]:
def sub(frame): #пишем функцию для подсчёта выручки
    if frame['subscription_type'] == 'ultra':
        return int(6 * frame['duration'] + 199)
    else:
        return int(50 * frame['count_rides'] + 8 * frame['duration'])

In [ ]:
#создаём датафрейм с агрегированными данными о поездках
data_pivot = data.groupby(['user_id', 'month']).agg({'distance' : 'sum', 'duration' : 'sum', 'subscription_type' : ['count', 'first']})
data_pivot.columns = ['distance', 'duration', 'count_rides', 'subscription_type']

In [ ]:
data_pivot['revenue'] = data_pivot.apply(sub, axis=1)
#находим выручку

In [ ]:
data_pivot #проверяем что получилось

In [ ]:
data_pivot.isna().sum() #проверяем получившуюся таблицу на пропуски

## Промежуточный вывод 
Был создан датафрейм с агрегированными данными о поездках на основе датафрейма с объединёнными данными из шага 4: было найдено суммарное расстояние, количество поездок и суммарное время **для каждого пользователя за каждый месяц**. Тем самым мы получили данные по каждому уникальному пользователю помесячно. В этот же датафрейм добавили столбец с помесячной выручкой, которую принёс каждый пользователь, обратившись к информации об условиях оплаты для подписчиков и тех, у кого нет подписки. Таким образом вследстие этого шага был создан датафрейм где для каждого пользователя помесячно известна **длительность поездок, расстояние, количество поездок, тип подписки и выручка**.

# Шаг 6. Проверка гипотез

**Определяем гипотезы:**
* Нулевая гипотеза "Пользователи с подпиской и без неё тратят одинаковое количество времени на поездки"
* Альтернативная гипотеза "Пользователи с подпиской тратят больше времени на поездки"

In [ ]:
alpha = 0.05 #уровень статистической значимости

results_h1 = st.ttest_ind(data_subs['duration'], data_unsubs['duration'], alternative='greater') 
# проводим ttest для двух выборок, в качестве альтернативной используем одностороннюю гипотезу «больше»

print(f'p-value: {results_h1.pvalue}')

if results_h1.pvalue < alpha: # проверяем p-value
    print('Отвергаем нулевую гипотезу: пользователи с подпиской и без неё тратят одинаковое количество времени на поездки')
else:
    print('Нет оснований отвергнуть нулевую гипотезу: пользователи с подпиской тратят больше времени на поездки')

Мы **отвергаем** гипотезу о том что пользователи с подпиской тратят одинаковое количество на поездки, так как значение p-value значительно меньше уровня статистической значимости, значит принимаем гипотезу о том что пользователи с подпиской тратят больше времени на поездки.

**Определяем гипотезы:**
* Нулевая гипотеза "Расстояние одной поездки пользователя с подпиской равняется 3130 м"
* Альтернативная гипотеза "Расстояние одной поездки значимо больше оптимального"

In [ ]:
interested_value = 3130 #оптимальное расстояние с точки зрения износа самоката

results_h2 = st.ttest_1samp(data_subs['distance'], interested_value, alternative='greater') 
#проводим ttest, в качестве альтернативной используем одностороннюю гипотезу «больше»

print('p-значение:', results_h2.pvalue)

if results_h2.pvalue < alpha: # проверяем p-value
    print('Отвергаем нулевую гипотезу: расстрояние одной поездки пользователя с подпиской равняется 3130 м')
else:
    print('Не получилось отвергнуть нулевую гипотезу: расстояние одной поездки значимо больше оптимального')

Мы **не можем отвергнуть** гипотезу о том что расстояние одной поездки равняется 3130 м, так как значение p-value значимо больше уровня статистической значимости, следовательно  расстояние одной поездки прибилизительно развняется 3130 м.

**Определяем гипотезы:**
* Нулевая гипотеза "Помесячная выручка подписчиков равна помесячной выручке пользователей без подписки"
* Альтернативная гипотеза "Помесячная выручка подписчиков выше помесячной выручки пользователей без подписки"

In [ ]:
results_h3 = st.ttest_ind(data_pivot[data_pivot['subscription_type'] == 'ultra']['revenue'], data_pivot[data_pivot['subscription_type'] == 'free']['revenue'], alternative='greater') 
# проводим ttest для двух выборок, в качестве альтернативной используем одностороннюю гипотезу «больше»

print(f'p-value: {results_h3.pvalue}')

if results_h3.pvalue < alpha: # проверяем p-value
    print('Отвергаем нулевую гипотезу: помесячная выручка подписчиков равна помесячной выручке пользователей без подписки')
else:
    print('Нет оснований отвергнуть нулевую гипотезу: помесячная выручка подписчиков выше помесячной выручки пользователей без подписки')

Мы **отвергаем** нулевую гипотезу о том что помесячная выручка от пользователей с подпиской равна помесячной выручке от пользователей без подписки, так как значение p-value значительно меньше уровня статистической значимости, следовательно мы можем принять альтернативную гипотезу о том что помесячная выручка подписчиков выше помесячной выручки пользователей без подписки.

Если у нас стоит задача **оценить стало ли меньше обращений в техническую поддержку после обновления** на основе файла с данными для каждого пользователя о количестве обращений до обновления и после него, нам необходимо использовать одностороннюю гипотезу для двух генеральных совокупностей. Нулевая гипотеза в этом случае будет звучать **"Между средними значениями этих двух совокупностей нет разницы, они равны"**, а альтернативная гипотеза будет звучать **"Среднее значение первой генеральной совокупности меньше среднего значения второй генеральной совокупности"**, при условии что за первую генеральную совокупность мы берём данные по обращениям после обновления. Для проверки этих гипотез мы будем использовать метод `ttest_rel()`, так как речь идёт об одной генеральной совокупности, с которой произошли некоторые изменения, с параметром **alternative** равным **'less'**. И если в этом случае **p-value** окажется меньше уровня статистической значимости - мы отвергнем нулевую гипотезу, иначе - оснований её отвергнуть у нас не будет.

## Промежуточный вывод
На данном этапе были проверены следующие гипотезы:
* Тратят ли пользователи с подпиской больше времени на поездки (гипотеза **принята** по результатам исследования с **p-value равным 1.2804098316645618e-34**)
* Не превышает ли расстояние, которое проезжают пользователи с подпиской за одну поездку, 3130 метров (гипотеза **не была отвергнута** по результатам исследования с **p-value равным 0.9195368847849785**)
* Будет ли помесячная выручка от пользователей с подпиской по месяцам выше, чем выручка от пользователей без подписки (гипотеза **подтверждена** по результатам исследования с **p-value равным 2.8130926188487286e-35**)
* Снизится ли количество обращений в тех. поддержку после обновления приложения (был построен план исследования, подобный предыдущим. Нулевая гипотеза в этом случае будет звучать **"Между средними значениями этих двух совокупностей нет разницы, они равны"**, а альтернативная гипотеза будет звучать **"Среднее значение первой генеральной совокупности меньше среднего значения второй генеральной совокупности"**, при условии что за первую генеральную совокупность мы берём данные по обращениям после обновления. Для проверки этих гипотез мы будем использовать метод `ttest_rel()`, так как речь идёт об одной генеральной совокупности, с которой произошли некоторые изменения, с параметром **alternative** равным **'less'**. И если в этом случае **p-value** окажется меньше уровня статистической значимости - мы отвергнем нулевую гипотезу, иначе - оснований её отвергнуть у нас не будет.

# Шаг 7. Распределения

In [ ]:
p = 0.1 #вероятность что пользователь продлит подписку
n = 1200 #сколько пользователей необходимо привлечь к акции
distr = [] #создаём список distr в котором будем хранить значения

for i in range (0, n+1):
    current_value = st.binom.pmf(i, n, p)
    distr.append(current_value)

plt.bar(range(0, n+1), distr)
plt.show()

mu = n * p # рассчитываем параметры нормального распределения
sigma = sqrt(n * p * (1 - p))

distr = st.norm(mu, sigma) # задаём нормальное распределение

result = distr.cdf(100) # найдите вероятность
result

Если разослать 1200 промокодов, риск получить менее 100 покупателей подписок крайне низок. На гистограмме видно, что в диапазоне от 0 до 100 количество значений минимально. Также аппроксимировав распределение нормальным, мы рассчитали вероятность получить менее 100 успехов - при значении n = 1000 вероятность равна 50%, при значении n = 1100 - 15.74%, а при значении n = 1200 - 2.71%, и это нам подходит.

In [ ]:
n = 1000000 #задаём параметры биномиального распределения
p = 0.4

mu = n * p #рассчитываем мат. ожидание нормального распределения
sigma = sqrt(n * p * (1 - p)) #рассчитываем стандартное отклонение нормального распределения

distr = st.norm(mu, sigma) #задаём нормальное распределение

result = distr.cdf(399500)
print(f'{result * 100} %')

Аппроксимировав биномиальное распределение нормальным и применив метод cdf() мы смогли определить что вероятность того, что уведомление откроют не более 399,5 тыс. пользователей равна приблизительно 15.4 %, что достаточно много, потому мы не можем быть уверены в таком количестве открытий уведомления.

## Промежуточный вывод
Для оценки минимального количества необходимых для акции промокодов было использовано биномиальное распределение и по графику, меняя значение **n** мы смогли определить что если разослать **1200 промокодов**, риск получить менее **100** покупателей подписок крайне низок. На гистограмме видно, что в диапазоне **от 0 до 100** количество значений минимально.
Для оценки вероятности того, что уведомление откроют не более **399,5 тыс.** пользователей было аппроксимировано нормальным биномиальное распределение. Применив к нему метод `cdf()` мы смогли определить что вероятность того, что уведомление откроют не более **399,5 тыс.** пользователей равна приблизительно **15.4 %**, что достаточно много, потому мы не можем быть уверены в таком количестве открытий уведомления.

# Общий вывод

В ходе данного исследования мы:
* Считали CSV-файлы с данными с помощью библиотеки **pandas** и сохранили их в датафреймы. Выведили первые строки каждого набора данных и изучили общую информацию о каждом датафрейме. В датафрейме **users** находятся данные о имени, возрасте, городе и типе подписки для каждого уникального идентификатора пользователя. В датафрейме **rides** находятся данные о расстоянии, длительности и дате поездки для каждого уникального идентификатора пользователя. В датафрейме **subscriptions** находятся данные о цене за минуту поездки, цене старта поездки, цене подписки для каждого типа подписки.
* Привели столбец **date** датафрейма **rides** к необходимому типу данных - **datetime**. Также создали новый столбец **month** на основе данных столбца **date**. Пропуски в данных найдены не были. Дубликаты были найдеты только в датафрейме **users** в количестве 31 и удалили их, так как их мало и они выглядят как баг системы. 
* Описали и визуализировали следующие данные: частота встречаемости городов (города распределены практически равномерно, данных по каждому городу **от 168 до 219 ед.** Наибольшее количество поездок приходится на **Пятигорск**, наименьшее - на **Москву**); соотношение пользователей с подпиской и без подписки (количество подписчиков сервиса около **700 ед.**, пользователей без подписки - немного больше **800 ед.**, они находятся в соотношении **45.27 % к 54.43 %** соответственно.); возраст пользователей (можно увидеть что большая часть значений возраста находится в области от **20 до 30**, среднее значение - **26**, минимальное - **12**, максимальное - **43**.); расстояние, которое пользователь преодолел за одну поездку (наибольшее количество поездок пользователей были на расстояние **3000 - 3500 м**, наименьшее - на расстояние более **5000 м**); продолжительность поездок (наибольшее количество поездок пользователей были длительностью **17-20 минут**, наименьшее - длительностью **до 5 минут и более 33 минут**).
* Объединили данные о пользователях, поездках и подписках в один датафрейм, воспользовавшись методом `merge()`. На основе этого датафрейма создали два новых - с данными о подписчиках и данными о пользователях без подписки, а также визуализировали информацию о расстоянии и времени поездок для пользователей обеих категорий. Самая частая длительность поездок подписчиков - **15-20 минут**, самая редкая - **до 7 минут и более 33 минут**. Самое частое расстояние поездок подписчиков - **3000-3500 м**, самое редкое - **до 2000 м и более 4500 м**. Самая частая длительность поездок пользователей без подписки - **14-21 минута**, самая редкая - **до 4 минут и более 31 минуты**. Самое частое расстояние поездок подписчиков - **2000-3500 м**, самое редкое - **более 5700 м**. Для обоих типов пользователей есть провисы в первой половине графика, но у пользователей с подпиской он находится на значении **1200-1800 м**, у пользователей без подписки - на значении **700-1500 м**. Относительно длительности - график для людей без подписки приближен к распределению по параболе, а графиц для людей с подпиской смещён влево.
* Создали датафрейм с агрегированными данными о поездках на основе датафрейма с объединёнными данными из шага 4: было найдено суммарное расстояние, количество поездок и суммарное время **для каждого пользователя за каждый месяц**. Тем самым мы получили данные по каждому уникальному пользователю помесячно. В этот же датафрейм добавили столбец с помесячной выручкой, которую принёс каждый пользователь, обратившись к информации об условиях оплаты для подписчиков и тех, у кого нет подписки. Таким образом вследстие этого шага был создан датафрейм где для каждого пользователя помесячно известна **длительность поездок, расстояние, количество поездок, тип подписки и выручка**.
* Проверили следующие гипотезы - тратят ли пользователи с подпиской больше времени на поездки (гипотеза **принята** по результатам исследования с **p-value равным 1.2804098316645618e-34**); не превышает ли расстояние, которое проезжают пользователи с подпиской за одну поездку, 3130 метров (гипотеза **не была отвергнута** по результатам исследования с **p-value равным 0.9195368847849785**); будет ли помесячная выручка от пользователей с подпиской по месяцам выше, чем выручка от пользователей без подписки (гипотеза **подтверждена** по результатам исследования с **p-value равным 2.8130926188487286e-35**); снизится ли количество обращений в тех. поддержку после обновления приложения (был построен план исследования, подобный предыдущим. Нулевая гипотеза в этом случае будет звучать **"Между средними значениями этих двух совокупностей нет разницы, они равны"**, а альтернативная гипотеза будет звучать **"Среднее значение первой генеральной совокупности меньше среднего значения второй генеральной совокупности"**, при условии что за первую генеральную совокупность мы берём данные по обращениям после обновления. Для проверки этих гипотез мы будем использовать метод `ttest_rel()`, так как речь идёт об одной генеральной совокупности, с которой произошли некоторые изменения, с параметром **alternative** равным **'less'**. И если в этом случае **p-value** окажется меньше уровня статистической значимости - мы отвергнем нулевую гипотезу, иначе - оснований её отвергнуть у нас не будет.
* Для оценки минимального количества необходимых для акции промокодов использовали биномиальное распределение и по графику, меняя значение **n** смогли определить что если разослать **1200 промокодов**, риск получить менее **100** покупателей подписок крайне низок. На гистограмме видно, что в диапазоне **от 0 до 100** количество значений минимально. Для оценки вероятности того, что уведомление откроют не более **399,5 тыс.** пользователей аппроксимировали нормальным биномиальное распределение. Применив к нему метод `cdf()` смогли определить что вероятность того, что уведомление откроют не более **399,5 тыс.** пользователей равна приблизительно **15.4 %**, что достаточно много, потому мы не можем быть уверены в таком количестве открытий уведомления.

**Изучив данные и рассмотрев все гипотезы, можно сделать вывод, что пользователи с подпиской являются более выгодными для компании чем пользователи без подписок, так как они тратят значительно больше времени на поездки и помесячная выручка по ним больше помесячной выручки от пользователей без подписки.**